    Instructions:
    1. Press Run All (or restart kernel and run all cells).
    2. You will be prompted to provide input values.


    Task 1: Implement a program which,
    – for each unique label l, computes the corresponding k latent semantics (of your choice) associated with the even
    numbered Caltec101 images, and
    – for the odd numbered images, predicts the most likely labels using distances/similarities computed under the
    label-specific latent semantics.
    The system should also output per-label precision, recall, and F1-score values as well as output an overall accuracy
    value.

In [1]:
FEATURE_SPACE = "resnet"

REDUCER = "LS1_resnet_118_svd_reducer"

In [2]:
from utils.database_utils import retrieve
from utils.vector_utils import get_latent_feature_vectors

reducer = retrieve(f'{REDUCER}.pt')

feature_vectors = retrieve(f'{FEATURE_SPACE}.pt')

latent_feature_vectors = get_latent_feature_vectors(feature_vectors, reducer)

In [3]:
from utils.vector_utils import get_representative_vectors_for_labels
from utils.dataset_utils import initialize_dataset

dataset = initialize_dataset()

rep_label_latent_vectors = get_representative_vectors_for_labels(latent_feature_vectors, dataset.categories, K=5)

from utils.vector_utils import rep_label_vectors_to_feature_vectors

rep_latent_feature_vectors = rep_label_vectors_to_feature_vectors(rep_label_latent_vectors)

Files already downloaded and verified


In [4]:
for label in rep_label_latent_vectors.keys():
    print(f"\nLabel: {label} k-latent semantics for even Caltech101 images:\n")
    for vector in rep_label_latent_vectors[label]:
        print(vector)


Label: Faces k-latent semantics for even Caltech101 images:

[ 1.81260794e+01  3.96978857e+00  3.96954209e+00  2.67866813e+00
  4.88306343e-01  1.65394723e+00 -1.97951595e+00 -4.32381093e-01
  5.13552534e-01  9.46524986e-01  1.98929411e-01  1.00334110e+00
 -1.00714983e+00  3.35429392e-01 -8.30365526e-02  8.25374317e-01
  2.97109968e-01  1.21522340e-01  9.86355913e-02  1.08108914e+00
 -1.11456958e+00  9.82190418e-01 -1.13119382e+00  3.38994722e-01
  6.04991219e-01 -8.68782633e-01 -1.14568984e+00 -1.07677144e-01
  1.28680346e+00  4.69585780e-02 -4.43487156e-01  9.72216777e-01
  7.51003270e-01 -1.12477149e+00  9.69734287e-01  2.01917513e-01
  8.27193566e-01  1.97198832e-01 -1.65564371e-01 -2.38930772e-01
  2.38071695e-01  7.69237954e-02  2.77834664e-01  5.26397568e-02
 -2.17798782e-01  6.27887442e-01 -4.83677074e-01 -4.70314558e-01
  8.48060347e-01  1.41708918e-01 -2.91331532e-01 -2.68900114e-01
 -4.68773402e-01  6.50515206e-01 -4.32997984e-01  9.20596424e-02
 -4.62379847e-01  2.80490249

In [5]:
odd_feature_vectors = retrieve(f'odd{FEATURE_SPACE}.pt')

odd_latent_semantics_vectors = reducer.reduce_features(odd_feature_vectors)

odd_latent_semantics = []
for (imgid, (label, _)), ls_vec in zip(odd_feature_vectors.items(), odd_latent_semantics_vectors):
    odd_latent_semantics.append((imgid, label, ls_vec))

In [6]:
# Strategy: Minimum distance to any cluster

from utils.distance_utils import top_k_unique_label_distance_ranker, cosine
from utils.perf_utils import PerLabelPerf

all_labels = dataset.categories
perf_calc = PerLabelPerf(all_labels)

for tuple in odd_latent_semantics:
    img_id, odd_label, odd_vector = tuple

    closest_dist = top_k_unique_label_distance_ranker(3, odd_vector, rep_latent_feature_vectors, cosine)

    perf_calc.process_perf(y=odd_label, y_pred=closest_dist[0][2])

    print(f"IMG_ID: {img_id}\tTrue label: {odd_label}\tPred labels: {closest_dist[0][2], closest_dist[1][2], closest_dist[2][2]}")

IMG_ID: 1	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'sea_horse')
IMG_ID: 3	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'panda')
IMG_ID: 5	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'dollar_bill')
IMG_ID: 7	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'strawberry')
IMG_ID: 9	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'dalmatian')
IMG_ID: 11	True label: Faces	Pred labels: ('Faces_easy', 'Faces', 'lobster')
IMG_ID: 13	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'electric_guitar')
IMG_ID: 15	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'Leopards')
IMG_ID: 17	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'strawberry')
IMG_ID: 19	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'dollar_bill')
IMG_ID: 21	True label: Faces	Pred labels: ('Faces', 'Faces_easy', 'cup')
IMG_ID: 23	True label: Faces	Pred labels: ('Faces_easy', 'Faces', 'brain')
IMG_ID: 25	True label: Faces	Pred labels: ('Faces', 'Faces_eas

In [7]:
from utils.query_input_processor import align_print

print("Per-label Precision, Recall, F1 Scores:\n")

max_label_len = max([len(label) for label in all_labels])
decimals = 4

for label in all_labels:
    precision = str(round(perf_calc.get_precision(label), decimals))
    recall = str(round(perf_calc.get_recall(label), decimals))
    f1_score = str(round(perf_calc.get_f1_score(label), decimals))
    print(f"Label: {align_print(label, max_label_len)}\tPrecision: {align_print(precision, decimals+2)}\tRecall: {align_print(recall, decimals+2)} \t F1 score:{align_print(f1_score, decimals+2)}\t")

print("\n\nOverall accuracy" + f": {perf_calc.get_overall_accuracy() * 100}%")

Per-label Precision, Recall, F1 Scores:

Label: Faces          	Precision: 1.0   	Recall: 0.9954 	 F1 score:0.9977	
Label: Faces_easy     	Precision: 1.0   	Recall: 0.9908 	 F1 score:0.9954	
Label: Leopards       	Precision: 0.9901	Recall: 1.0    	 F1 score:0.995 	
Label: Motorbikes     	Precision: 0.9925	Recall: 0.99   	 F1 score:0.9912	
Label: accordion      	Precision: 1.0   	Recall: 1.0    	 F1 score:1.0   	
Label: airplanes      	Precision: 0.9874	Recall: 0.9775 	 F1 score:0.9824	
Label: anchor         	Precision: 0.5238	Recall: 0.5238 	 F1 score:0.5238	
Label: ant            	Precision: 0.6667	Recall: 0.8571 	 F1 score:0.75  	
Label: barrel         	Precision: 0.88  	Recall: 0.9167 	 F1 score:0.898 	
Label: bass           	Precision: 0.84  	Recall: 0.7778 	 F1 score:0.8077	
Label: beaver         	Precision: 0.7407	Recall: 0.8696 	 F1 score:0.8   	
Label: binocular      	Precision: 0.875 	Recall: 0.875  	 F1 score:0.875 	
Label: bonsai         	Precision: 0.9385	Recall: 0.9531 	 F